# Install all dependencies

In [ ]:
!pip install torch transformers boto3

# Create AWS S3 client

In [2]:
import boto3
import json
import os

try:
    from google.colab import userdata
    AWS_ACCESS_KEY = userdata.get('aws_access_key_id')
    AWS_SECRET_ACCESS_KEY = userdata.get('aws_secret_access_key')
    BUCKET = userdata.get('bucket')
    s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
except ImportError:
    BUCKET = os.environ.get('BUCKET')
    s3 = boto3.client('s3')



# Available CPU Cores

In [4]:
import os
cpu_count = os.cpu_count()
cpu_count

8

# Setup Unixcoder Model

In [5]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.

import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig

class UniXcoder(nn.Module):
    def __init__(self, model_name):
        """
            Build UniXcoder.

            Parameters:

            * `model_name`- huggingface model card name. e.g. microsoft/unixcoder-base
        """
        super(UniXcoder, self).__init__()
        self.tokenizer = RobertaTokenizer.from_pretrained(model_name)
        self.config = RobertaConfig.from_pretrained(model_name)
        self.config.is_decoder = True
        self.model = RobertaModel.from_pretrained(model_name, config=self.config)

        self.register_buffer("bias", torch.tril(torch.ones((1024, 1024), dtype=torch.uint8)).view(1,1024, 1024))
        self.lm_head = nn.Linear(self.config.hidden_size, self.config.vocab_size, bias=False)
        self.lm_head.weight = self.model.embeddings.word_embeddings.weight
        self.lsm = nn.LogSoftmax(dim=-1)

        self.tokenizer.add_tokens(["<mask0>"],special_tokens=True)

    def tokenize(self, inputs, mode="<encoder-only>", max_length=512, padding=False):
        """
        Convert string to token ids

        Parameters:

        * `inputs`- list of input strings.
        * `max_length`- The maximum total source sequence length after tokenization.
        * `padding`- whether to pad source sequence length to max_length.
        * `mode`- which mode the sequence will use. i.e. <encoder-only>, <decoder-only>, <encoder-decoder>
        """
        assert mode in ["<encoder-only>", "<decoder-only>", "<encoder-decoder>"]
        assert max_length < 1024

        tokenizer = self.tokenizer

        tokens_ids = []
        for x in inputs:
            tokens = tokenizer.tokenize(x)
            if mode == "<encoder-only>":
                tokens = tokens[:max_length-4]
                tokens = [tokenizer.cls_token,mode,tokenizer.sep_token] + tokens + [tokenizer.sep_token]
            elif mode == "<decoder-only>":
                tokens = tokens[-(max_length-3):]
                tokens = [tokenizer.cls_token,mode,tokenizer.sep_token] + tokens
            else:
                tokens = tokens[:max_length-5]
                tokens = [tokenizer.cls_token,mode,tokenizer.sep_token] + tokens + [tokenizer.sep_token]

            tokens_id = tokenizer.convert_tokens_to_ids(tokens)
            if padding:
                tokens_id = tokens_id + [self.config.pad_token_id] * (max_length-len(tokens_id))
            tokens_ids.append(tokens_id)
        return tokens_ids

    def decode(self, source_ids):
        """ Convert token ids to string """
        predictions = []
        for x in source_ids:
            prediction = []
            for y in x:
                t = y.cpu().numpy()
                t = list(t)
                if 0 in t:
                    t = t[:t.index(0)]
                text = self.tokenizer.decode(t,clean_up_tokenization_spaces=False)
                prediction.append(text)
            predictions.append(prediction)
        return predictions

    def forward(self, source_ids):
        """ Obtain token embeddings and sentence embeddings """
        mask = source_ids.ne(self.config.pad_token_id)
        token_embeddings = self.model(source_ids,attention_mask = mask.unsqueeze(1) * mask.unsqueeze(2))[0]
        sentence_embeddings = (token_embeddings * mask.unsqueeze(-1)).sum(1) / mask.sum(-1).unsqueeze(-1)
        return token_embeddings, sentence_embeddings

    def generate(self, source_ids, decoder_only = True, eos_id = None, beam_size = 5, max_length = 64):
        """ Generate sequence given context (source_ids) """

        # Set encoder mask attention matrix: bidirectional for <encoder-decoder>, unirectional for <decoder-only>
        if decoder_only:
            mask = self.bias[:,:source_ids.size(-1),:source_ids.size(-1)]
        else:
            mask = source_ids.ne(self.config.pad_token_id)
            mask = mask.unsqueeze(1) * mask.unsqueeze(2)

        if eos_id is None:
            eos_id = self.config.eos_token_id

        device = source_ids.device

        # Decoding using beam search
        preds = []
        zero = torch.LongTensor(1).fill_(0).to(device)
        source_len = list(source_ids.ne(1).sum(-1).cpu().numpy())
        length = source_ids.size(-1)
        encoder_output = self.model(source_ids,attention_mask=mask)
        for i in range(source_ids.shape[0]):
            context = [[x[i:i+1,:,:source_len[i]].repeat(beam_size,1,1,1) for x in y]
                     for y in encoder_output.past_key_values]
            beam = Beam(beam_size,eos_id,device)
            input_ids = beam.getCurrentState().clone()
            context_ids = source_ids[i:i+1,:source_len[i]].repeat(beam_size,1)
            out = encoder_output.last_hidden_state[i:i+1,:source_len[i]].repeat(beam_size,1,1)
            for _ in range(max_length):
                if beam.done():
                    break
                if _ == 0:
                    hidden_states = out[:,-1,:]
                    out = self.lsm(self.lm_head(hidden_states)).data
                    beam.advance(out)
                    input_ids.data.copy_(input_ids.data.index_select(0, beam.getCurrentOrigin()))
                    input_ids = beam.getCurrentState().clone()
                else:
                    length = context_ids.size(-1)+input_ids.size(-1)
                    out = self.model(input_ids,attention_mask=self.bias[:,context_ids.size(-1):length,:length],
                                       past_key_values=context).last_hidden_state
                    hidden_states = out[:,-1,:]
                    out = self.lsm(self.lm_head(hidden_states)).data
                    beam.advance(out)
                    input_ids.data.copy_(input_ids.data.index_select(0, beam.getCurrentOrigin()))
                    input_ids = torch.cat((input_ids,beam.getCurrentState().clone()),-1)
            hyp = beam.getHyp(beam.getFinal())
            pred = beam.buildTargetTokens(hyp)[:beam_size]
            pred = [torch.cat([x.view(-1) for x in p]+[zero]*(max_length-len(p))).view(1,-1) for p in pred]
            preds.append(torch.cat(pred,0).unsqueeze(0))

        preds = torch.cat(preds,0)

        return preds



class Beam(object):
    def __init__(self, size, eos, device):
        self.size = size
        self.device = device
        # The score for each translation on the beam.
        self.scores = torch.FloatTensor(size).zero_().to(device)
        # The backpointers at each time-step.
        self.prevKs = []
        # The outputs at each time-step.
        self.nextYs = [torch.LongTensor(size).fill_(0).to(device)]
        # Has EOS topped the beam yet.
        self._eos = eos
        self.eosTop = False
        # Time and k pair for finished.
        self.finished = []

    def getCurrentState(self):
        "Get the outputs for the current timestep."
        batch = self.nextYs[-1].view(-1, 1)
        return batch

    def getCurrentOrigin(self):
        "Get the backpointers for the current timestep."
        return self.prevKs[-1]

    def advance(self, wordLk):
        """
        Given prob over words for every last beam `wordLk` and attention
        `attnOut`: Compute and update the beam search.

        Parameters:

        * `wordLk`- probs of advancing from the last step (K x words)
        * `attnOut`- attention at the last step

        Returns: True if beam search is complete.
        """
        numWords = wordLk.size(1)

        # Sum the previous scores.
        if len(self.prevKs) > 0:
            beamLk = wordLk + self.scores.unsqueeze(1).expand_as(wordLk)

            # Don't let EOS have children.
            for i in range(self.nextYs[-1].size(0)):
                if self.nextYs[-1][i] == self._eos:
                    beamLk[i] = -1e20
        else:
            beamLk = wordLk[0]
        flatBeamLk = beamLk.view(-1)
        bestScores, bestScoresId = flatBeamLk.topk(self.size, 0, True, True)

        self.scores = bestScores

        # bestScoresId is flattened beam x word array, so calculate which
        # word and beam each score came from
        prevK = torch.div(bestScoresId, numWords, rounding_mode="floor")
        self.prevKs.append(prevK)
        self.nextYs.append((bestScoresId - prevK * numWords))


        for i in range(self.nextYs[-1].size(0)):
            if self.nextYs[-1][i] == self._eos:
                s = self.scores[i]
                self.finished.append((s, len(self.nextYs) - 1, i))

        # End condition is when top-of-beam is EOS and no global score.
        if self.nextYs[-1][0] == self._eos:
            self.eosTop = True

    def done(self):
        return self.eosTop and len(self.finished) >= self.size

    def getFinal(self):
        if len(self.finished) == 0:
            self.finished.append((self.scores[0], len(self.nextYs) - 1, 0))
        self.finished.sort(key=lambda a: -a[0])
        if len(self.finished) != self.size:
            unfinished=[]
            for i in range(self.nextYs[-1].size(0)):
                if self.nextYs[-1][i] != self._eos:
                    s = self.scores[i]
                    unfinished.append((s, len(self.nextYs) - 1, i))
            unfinished.sort(key=lambda a: -a[0])
            self.finished+=unfinished[:self.size-len(self.finished)]
        return self.finished[:self.size]

    def getHyp(self, beam_res):
        """
        Walk back to construct the full hypothesis.
        """
        hyps=[]
        for _,timestep, k in beam_res:
            hyp = []
            for j in range(len(self.prevKs[:timestep]) - 1, -1, -1):
                hyp.append(self.nextYs[j+1][k])
                k = self.prevKs[j][k]
            hyps.append(hyp[::-1])
        return hyps

    def buildTargetTokens(self, preds):
        sentence=[]
        for pred in preds:
            tokens = []
            for tok in pred:
                if tok==self._eos:
                    break
                tokens.append(tok)
            sentence.append(tokens)
        return sentence

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#https://huggingface.co/Lazyhope/unixcoder-clone-detection/tree/main
model = UniXcoder("Lazyhope/unixcoder-clone-detection")
model.to(device)

/Users/jbenitezg/Documents/GitHub/ECI-MS-Thesis/clone-detection/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


UniXcoder(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(51416, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768, padding_idx=1)
      (token_type_embeddings): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

# Setup metadata

In [6]:
response = s3.get_object(Bucket=BUCKET, Key='metadata.json')
metadata = json.loads(response['Body'].read().decode('utf-8'))

# Setup completed repositories
All completed repositories are stored in the `completed_repositories` set.

In [7]:
def clone_detected():
    response = s3.get_object(Bucket=BUCKET, Key='execution.json')
    urls = json.loads(response['Body'].read().decode('utf-8'))
    return [url for url in urls]

def save_clone_detected(clone_detected):
    s3.put_object(Bucket=BUCKET, Key='execution.json', Body=json.dumps(clone_detected))

def skipped():
    response = s3.get_object(Bucket=BUCKET, Key='execution.json')
    urls = json.loads(response['Body'].read().decode('utf-8'))
    return [url for url in urls]

def save_skipped(skipped):
    s3.put_object(Bucket=BUCKET, Key='execution.json', Body=json.dumps(skipped))

repositories_completed = clone_detected()
skipped = skipped()
print(f"Processed: {len(repositories_completed)}/{len(metadata)}")

Processed: 0/15841


# Code Similarity Function

In [9]:
# Cosine similarity between two functions
def code_similarity(func1, func2, cache):
  norm_f_func1_embedding = cache[func1['id']]
  norm_func2_embedding = cache[func2['id']]

  f_func_nl_similarity = torch.einsum("ac,bc->ab",norm_f_func1_embedding,norm_func2_embedding)

  return f_func_nl_similarity, func1['id'], func2['id']


# Detecting Code Clones in Repositories

In [ ]:
import itertools

def save_similarities(path, similarities):
    print(f"Saving similarities for {path}")
    s3.put_object(Bucket=BUCKET, Key=f"{path}/similarities.json", Body=json.dumps(similarities))

def filter_length(f1, f2, threshold=0.8):
    length_ratio = len(f1['code']) / len(f2['code'])
    return length_ratio >= threshold

def is_test(func):
    if 'test' in func['path']:
        return True
    return False

def get_dupl_paths(key):
    response = s3.get_object(Bucket=BUCKET, Key=f"{key}/dupl_files.json")
    return json.loads(response['Body'].read().decode('utf-8'))

def is_on_dupl_paths(func, dupl_paths):
    for path in dupl_paths:
        if path == "":
            continue
        if path in func['path']:
            return True
    return False

def surpass_limit(iter, limit=100_000):
    count = 0
    for i in iter:
        count += 1
        if count >= limit:
            return True
    return False

def get_functions(key):
    response = s3.get_object(Bucket=BUCKET, Key=key)
    raw_functions = response['Body'].read().decode('utf-8')
    return json.loads(raw_functions)

def filter_functions(functions_obj, key):
    no_tests = [func for func in functions_obj if not is_test(func)]
    dupl_paths = get_dupl_paths(key)
    in_dupl_paths = [func for func in no_tests if is_on_dupl_paths(func, dupl_paths)]
    if surpass_limit(itertools.combinations(in_dupl_paths, 2)):
        print(f"Too many combinations for {key}")
        return []
    raw_combinations = itertools.combinations(in_dupl_paths, 2)
    filtered_combinations = [combination for combination in raw_combinations if filter_length(combination[0], combination[1])]
    if surpass_limit(filtered_combinations):
        print(f"Too many filtered combinations for {key}")
        return []
    return filtered_combinations, in_dupl_paths

def pre_process_functions(functions_obj):
    cache = {}
    for function in functions_obj:
        f_id = function['id']
        # save normalized function embedding
        code = function['code']
        tokens_ids = model.tokenize([code],max_length=512,mode="<encoder-only>")
        source_ids = torch.tensor(tokens_ids).to(device)
        tokens_embeddings,f_func_embedding = model(source_ids)
        norm_f_func_embedding = torch.nn.functional.normalize(f_func_embedding, p=2, dim=1)
        cache[f_id] = norm_f_func_embedding
    return cache

In [ ]:

def process_repository(repository):
    repo_path = f"{repository['owner']}/{repository['name']}"
    print(f"Processing {repo_path}")
    path = f"{repo_path}/go-functions.json"
    functions_obj = get_functions(path)
    try:
        filtered_combinations, filtered_functions_obj = filter_functions(functions_obj, repo_path)
        if len(filtered_combinations) == 0:
            skipped.append(repository['url'])
            save_skipped(skipped)
            print(f"Skipped {repo_path}")
            return
        print(f"Processing {len(filtered_combinations)} filtered combinations for {repo_path}")
        cache = pre_process_functions(filtered_functions_obj)
        similarities = []
        for c1, c2 in filtered_combinations:
            similarity, id1, id2 = code_similarity(c1, c2, cache)
            similarities.append({'similarity': similarity.item(), 'id1': id1, 'id2': id2})
        save_similarities(repo_path, similarities)
        repositories_completed.append(repository['url'])
        save_clone_detected(repositories_completed)
        print(f"Finished processing {repo_path}")
    except Exception as e:
        print(f"Error processing {repo_path}: {e}")
    

missing = [repository for repository in metadata if repository['url'] not in repositories_completed and repository['url'] not in skipped]
print("Missing repositories:", len(missing))
for repository in missing:
    process_repository(repository)


Processing patrickmn/go-cache
Total combinations: 15753
Filtered combinations: 12145


KeyboardInterrupt: 